[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BRomans/UnicornPythonEssentialsToolkit/blob/main/notebooks/P300_Epoching.ipynb)



In [ ]:
!pip install mne # Use only on Google Colab

In [30]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from utils.loader import load_data, unicorn_channels, convert_to_mne
from brainflow import BoardIds, BoardShim, BrainFlowInputParams
from utils.layouts import layouts

#%matplotlib inline
matplotlib.use("Qt5Agg")

path_data = '../data'
path_mount = ''

In [42]:
board_id = BoardIds.ENOPHONE_BOARD
board = BoardShim(board_id, BrainFlowInputParams())
fs = board.get_sampling_rate(board_id)
chs = layouts[board_id]['channels']

eeg, trigger, dataframe = load_data("../export/session.csv", board=board_id, header='eno', fs=fs, skiprows=5, delimiter='\t')
plt.plot(trigger)
plt.show()
dataframe

,Sample,A1,C3,C4,A2,Time,Trigger
0,1247.0,577.151775,472.342968,384.944677,358.504057,1.716155e+09,0.0
1,1248.0,-172.972679,-287.342072,-319.695473,-333.923101,1.716155e+09,0.0
2,1249.0,-558.108091,-707.405806,-745.385885,-759.750605,1.716155e+09,0.0
3,1250.0,8.606911,-153.124332,-252.485275,-278.759003,1.716155e+09,0.0
4,1251.0,800.877810,664.889812,536.781549,504.618883,1.716155e+09,0.0
...,...,...,...,...,...,...,...
2251,3498.0,-705.760717,-735.604763,-730.949640,-685.083866,1.716155e+09,0.0
2252,3499.0,189.119577,162.535906,90.926886,123.447180,1.716155e+09,0.0
2253,3500.0,806.784630,822.514296,748.145580,780.624151,1.716155e+09,0.0
2254,3501.0,379.514694,421.363115,407.713652,453.108549,1.716155e+09,0.0


#### Convert to MNE format
MNE is a popular Python library for EEG data analysis. It provides a simple and intuitive way to work with EEG data, and it is widely used in the scientific community. The library provides many useful tools for EEG data analysis, such as data visualization, signal processing, and machine learning. In this section, we will convert the EEG data to the MNE format, so that we can use the library to analyze the data.
Documentation: [MNE](https://mne.tools/stable/index.html)

In [40]:
# Convert to MNE format
raw_data = convert_to_mne(eeg, trigger, fs=fs, chs=chs, recompute=False) # recompute=True to recalculate the event labels if the values are negative

Creating RawArray with float64 data, n_channels=4, n_times=78
    Range : 0 ... 77 =      0.000 ...     0.308 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=78
    Range : 0 ... 77 =      0.000 ...     0.308 secs
Ready.


In [41]:
# Compute PSD
Pxx = raw_data.compute_psd(fmin=0, fmax=fs/2)
Pxx.plot()
plt.show()

Effective window size : 0.312 (s)


In [22]:
filtered = raw_data.copy() # the method filters the signal in-place, so this time I
                      # want to preserve the original signal and filter just a
                      # temporary copy of it

# remove power line noise
filtered.notch_filter(50) 
filtered.notch_filter(60) 
# Apply band-pass filtering
filtered.filter(1,40) 

pxx_filt = filtered.compute_psd(fmin=0, fmax=50)
pxx_filt.plot()
plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband e

### Data Analysis


1. Mark the events with custom labels 
2. Epoching and baseline correction
3. Extract EEG data from epoch and define baseline segment.
4. Compute average P300 across epochs and channels.
5. Visualize ERPs for each channel.
6. Compute time-frequency representation using Morlet wavelets.
7. Compute power spectral density (PSD) and plot topographic map.

#### 1. Mark the events with custom labels
To extract events we need to provide the raw data and the name of the channel that contains the trigger information. The function `find_events()` will return a numpy array with the events and the event ids. We can also assign colors to the events to visualize them in the plot.

In [ ]:
from mne import find_events

ev_ids = {'NT': 2, 'T':1}
event_colors = {2:'r', 1:'g'}
stim_channel = 'STI'
events = find_events(filtered, stim_channel=stim_channel)
filtered.plot(events=events, event_id=ev_ids, event_color=event_colors, color = 'Gray', block = True, clipping=None, scalings=25e-6)
plt.show()

#### 2.Epoching and baseline correction
The next step is to create the epochs. We need to provide the raw data, the events, and the event ids. We can also specify the time window for the epochs and the baseline correction. `tmin` and `tmax`  parameters define the time window extracted as single epoch relatively to the single event, `baseline` is the timespan from which the data for the baseline correction is extracted.The function `Epochs()` will return an object with the epochs.

In [ ]:
# creating the Epochs object combining EEG data and events info
from mne import Epochs

eps = Epochs(filtered, events, event_id=ev_ids,
             tmin=-.6, tmax=0.8, baseline=(-.6,-.1))
            
eps.plot(block=True)# .plot() method for Epoch objects has not clipping parameter
plt.show()

#### 3.Compute average P300 across epochs. 
The next step is to compute the average P300 across epochs. We can choose the target and the baseline period. The function `average()` will return an object with the average P300 across epochs.

In [ ]:
# Choose target
target = "T"

# Baseline period
baseline=(-.6,-.1)

# Average epochs across specified channels
average_channels = eps[target].average(picks=unicorn_channels)
average_channels.apply_baseline(baseline)
fig = average_channels.plot_joint(times=[-0.4, -0.3, 0.05, 0.1, 0.2, 0.3, 0.4, 0.6], title="Average P300 across epochs", show=False)
fig.show()

#### 4. Visualize ERPs for each channel
The next step is to visualize the ERPs for each channel. We can choose the target and the baseline period. The function `plot()` will return a plot with the ERPs for each channel. We can also add vertical colored lines to mark the time windows of interest.

In [ ]:
for channel in unicorn_channels[0:3]:
  average_epochs = eps[target].average(method="mean")
  average_epochs.apply_baseline(baseline)
  try:
      fig = average_epochs.plot(picks=channel, titles="Average P300 - " + channel, show=False)
      axes = fig.get_axes()
    
      for ax in axes:
          ax.axvspan(0.12, 0.17, color='lightblue', label="N100", alpha=0.5)
          ax.axvspan(0.17, 0.21, color='orange', label="P200", alpha=0.5)
          ax.axvspan(0.21, 0.25, color='blue', label="N200", alpha=0.3)
          ax.axvspan(0.25, 0.4, color='red', label="P300", alpha=0.3)
      fig.legend(loc='upper right', frameon=True)
      fig.set_size_inches(10, 5)
    
      # Show the plot
      fig.show()
  except Exception as e:
      print(e)

#### 5. Compute time-frequency representation using Morlet wavelets.
An alternative way to visualize the P300 is to compute the time-frequency representation using Morlet wavelets. We can choose the target and the frequency range. The function `tfr_morlet()` will return an object with the time-frequency representation.

In [ ]:
from mne.time_frequency import tfr_morlet
#Choose target
target = "T"

# create a time-frequency representation using Morlet wavelets
freqs = np.logspace(*np.log10([6, 35]), num=8)
tfr, itc = tfr_morlet(eps[target], freqs, n_cycles=5, average=True, return_itc=True)

# Apply baseline correction using mean power in the time range of interest
baseline=(-.6,-.1)
tfr.apply_baseline(baseline, mode='mean')

for channel in unicorn_channels[0:3]:
    try:
        tfr.plot(mode='zlogratio', picks=channel, fmin=1, fmax=20, cmap='viridis', title=channel)
        plt.show()
    except Exception as e:
        print(e)

#### 6. Compute power spectral density (PSD) and plot topographic map.
The last step is to compute the power spectral density (PSD) and plot the topographic map. We can choose the target and the frequency range. The function `compute_psd()` will return an object with the PSD. We can also plot the topographic map using the function `plot_topomap()`. The topographic map will show the distribution of the power across the scalp in each frequency band.

In [ ]:
epo_spectrum = eps["T"].compute_psd()
psds, freqs = epo_spectrum.get_data(return_freqs=True)
print(f'\nPSDs shape: {psds.shape}, freqs shape: {freqs.shape}')
evoked = eps["T"].average()
evk_spectrum = evoked.compute_psd()
fig = evk_spectrum.plot_topomap(ch_type='eeg', agg_fun=np.median)
fig.show()